In [1]:
import jsonpickle
import numpy as np
import mpmath
from mpmath import iv
from symbolic.cartpole_abstract import CartPoleEnv_abstract
from verification_runs.cartpole_bab_load import generateCartpoleDomainExplorer

In [2]:
with open("../save/safe_domains.json", 'r') as f:
    frozen_safe = jsonpickle.decode(f.read())
# with open("../save/unsafe_domains.json", 'r') as f:
#     frozen_unsafe = jsonpickle.decode(f.read())
# with open("../save/ignore_domains.json", 'r') as f:
#     frozen_ignore = jsonpickle.decode(f.read())
frozen_safe = np.stack(frozen_safe).take(range(10), axis=0)
# frozen_unsafe = np.stack(frozen_unsafe)  # .take(range(10), axis=0)
# frozen_ignore = np.stack(frozen_ignore)  # .take(range(10), axis=0)

In [3]:
env = CartPoleEnv_abstract()

In [4]:
def interval_unwrap(state):
    unwrapped_state = tuple([[float(x.a), float(x.b)] for x in state])
    return unwrapped_state


In [5]:
def step_state(state, action):
    env.reset()
    env.state = state
    next_state, _, _, _ = env.step(action)
    return tuple(next_state)


In [6]:
next_states = []
for interval in frozen_safe:
    state = tuple([iv.mpf([x.item(0), x.item(1)]) for x in interval])
    next_state = step_state(state, 1)
    unwrapped_next_state = interval_unwrap(next_state)
    next_states.append(unwrapped_next_state)

In [7]:
for i in next_states:
    print(i)

next_states_array = np.array(next_states,dtype=np.float32) #turns the list in an array


([0.015, 0.52], [0.9284048477265038, 1.1819933783687089], [0.7549999999999999, 1.01], [0.25065339497650124, 0.603794661498451])
([0.015, 0.52], [0.9285242866059016, 1.1822390070806543], [0.7599999999999999, 1.0150000000000001], [0.5003856966783815, 0.8536969287055857])
([0.015, 0.52], [0.9287233514048978, 1.182582887277378], [0.7649999999999999, 1.02], [0.750010919061014, 1.1035340407174767])
([0.015, 0.27], [0.9408294539596576, 1.195137036826784], [0.0, 0.255], [-0.29270555524017594, 0.04539122146617208])
([0.015, 0.27], [0.9359604666797062, 1.1908585568813368], [0.25, 0.505], [-0.20465107580731254, 0.1461576142109768])
([0.015, 0.27], [0.9408294539596576, 1.195182293648599], [0.005, 0.26], [-0.04277344047289816, 0.2953912214661721])
([0.015, 0.27], [0.935975302770512, 1.1909458656463738], [0.25499999999999995, 0.51], [0.04522203237223449, 0.3961380843691102])
([0.015, 0.27], [0.9317823840697175, 1.1860013425596678], [0.5, 0.755], [-0.1038961937282073, 0.2508974089737108])
([0.2649999

In [8]:
explorer, verification_model = generateCartpoleDomainExplorer()


Loading complete


In [9]:
explorer.explore(verification_model, next_states_array, precision=1e-6, min_area=0)
safe_next = explorer.safe_domains
unsafe_next = explorer.unsafe_domains
ignore_next = explorer.ignore_domains

2019-10-08 14:34:35,311	WARNING worker.py:1438 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-10-08 14:34:35,317	INFO resource_spec.py:205 -- Starting Ray with 10.94 GiB memory available for workers and up to 5.49 GiB for objects. You can adjust these settings with ray.remote(memory=<bytes>, object_store_memory=<bytes>).
2019-10-08 14:34:36,383	WARNING worker.py:351 -- WARNING: Falling back to serializing objects of type <class 'torch.nn.backends.thnn.THNNFunctionBackend'> by using pickle. This may be inefficient.


queue length : 20, # safe domains: 0, abstract areas: [unknown:100.000% --> safe:0.000%, unsafe:0.000%, ignore:0.000%], shortest_dim:0.2549999952316284, min_area:0.002795(pid=3710) Academic license - for non-commercial use only
(pid=3705) Academic license - for non-commercial use only
(pid=3711) Academic license - for non-commercial use only
(pid=3707) Academic license - for non-commercial use only
(pid=3706) Academic license - for non-commercial use only
(pid=3708) Academic license - for non-commercial use only
(pid=3704) Academic license - for non-commercial use only
(pid=3709) Academic license - for non-commercial use only
queue length : 0, # safe domains: 20, abstract areas: [unknown:92.500% --> safe:7.500%, unsafe:0.000%, ignore:0.000%], shortest_dim:0.2549999952316284, min_area:0.002795

In [23]:
frozen_safe = jsonpickle.encode([i for i in next_states_array])
with open("../save/next_safe_domains.json", 'w+') as f:
    f.write(frozen_safe)
